# GND/Getty Semi-Auto Updater

UNIDAM instance of Easydb5 at the University of Vienna

Trying to automate `Kuenstler` objecttype insertion of GND entries

Author: Janos Bekesi (janos.bekesi@univie.ac.at)

## Preconditions

Some grasp of `python` and [pandas](https://pandas.pydata.org/) is needed for anyone using this notebook, since some small code changes will probably be necessary.

Also beforehand, the `easydb` datamodel has to be changed accordingly to host GND, Getty and Wikidata entries.


In [ ]:
from pathlib import Path
from pprint import pprint
from zipfile import ZipFile
import csv
import time
import datetime
import json

# external libraries:
# they should be installed by `pip` or `conda` in the notebook environment (e.g. jupyterhub)
import pandas as pd
import request
import duckdb
import tqdm

datadir = Path("./data-unidam")
pprint(sorted(datadir.iterdir())[:10])

# inserting data from excel file (taxons acceptet etc)
def to_csv(df, csvname, headers=None):
    if headers:
        df.loc[-2] = headers
    df.index = df.index + 1  # shifting index
    df = df.sort_index()  # sorting by index 
    df.to_csv(datadir.joinpath("{}.csv".format(csvname)), quoting=2, sep=";", index=False)  
    
def curtime(long=False):
    now = datetime.datetime.now()
    if long:
        generated_at = now.isoformat()
        generated_at = generated_at.split(".")[0] #  . + '}""'
        return long
    return now.strftime("%Y%m%d_%H%M")

## Dataset

Create a `pandas` dataset from an CSV export of all `kuenstler` objects

The datafile has to be in `datadir` (which could be adopted to respective needs)

In [58]:
datafile = "unidam_kuenstler.zip"
with ZipFile(str(datadir.joinpath(datafile) )) as zf:
    for fn in zf.namelist():
        if fn.endswith(".csv"):
            with zf.open(fn) as csvr:
                df = pd.read_csv(csvr, sep=";")

print(f"{df.shape[0]} rows")

13926 rows


## GND search function definition

Wikidata entries are retained, too

queries GND via `lobid.org` (cf. easydb custom data type `gnd`)

In [112]:
def query_gnd(
    person_name,
    upper_limit=100,
    restrict_by_birthyear=False,
    write_rdf=False,
    write_csv=False,
    birthyear_max=1970,
    lebensdaten="",
    verbose=False,
    ):
    lobid = "https://lobid.org/gnd/search"
    pn = "preferredName:{}"
    params = {"filter": "type:Person", "format": "json"}
    uris = []
    no_rec = 0
    params["q"] = pn.format(person_name)
    r = requests.get(lobid, params=params)
    if r.status_code != 200:
        print(r.status_code, "status")
        return 0
    data = r.json()
    items = int(data["totalItems"])
    if int(items) == 0:
        if verbose:
            print(f"   nothing found in GND for {person_name}")  #  = no bindings
    elif items > upper_limit:
        print(
            f"{items} are more than {upper_limit} GND entries ({no_rec}) found for {person_name}"
        )  #  = no bindings
        return items
    for m in data.get("member", []):
        date_birth = m.get("dateOfBirth", None)
        date_death = m.get("dateOfDeath", None)
        pref_name = m.get("preferredName")
        if not pref_name == person_name:
            continue
        gnd_id = m.get("gndIdentifier")
        gnd_url = m.get("id")
        sameas = m.get("sameAs", [])
        wiki_id = ""
        if verbose:
            print(gnd_url, gnd_id, pref_name, date_birth, date_death)
        try:
            if not date_birth:
                dob = "0"
            else:
                if isinstance(date_birth, list):
                    date_birth = date_birth[0]
                dob = (
                    date_birth
                    if date_birth.find("-") == -1
                    else date_birth.split("-")[0]
                )
            dob = int(dob)
        except ValueError:
            dob = 0
        if restrict_by_birthyear and dob > birthyear_max:
            continue
        if lebensdaten: #  try to confirm:
            if lebensdaten.find(str(dob)) < 0:
                continue
        for sa in sameas:
            coll = sa.get("collection", {})
            # print("-" * 40)
            if coll.get("abbr", "") == "WIKIDATA":
                wiki_id = sa.get("id", "")
                # print("  wikidata:", wiki_id)
                break
        uris.append((pref_name, dob, gnd_url, wiki_id))
    if write_csv:
        uris = [("dateofbirth", "uri", "wiki_uri")] + uris
        with open(filename.with_suffix(".csv"), "w") as urifile:
            csvw = csv.writer(urifile)
            csvw.writerows(uris)
    else:
        return uris

## Getty search function

In [1]:
   
def query_getty(search_name, bio, voc="ulan", max_result=4, verbose=True):
    """
    https://ws.gbv.de/suggest/getty/?searchstring=' + getty_searchterm + '&voc=' + 
    getty_searchtype + '&count=' + getty_countSuggestion"
    
    """
    getty_url = "https://ws.gbv.de/suggest/getty"
    params = {"searchstring": "", "count": max_result, "voc": voc}
    perscon = "PersonConcept"
    uris = []
    no_rec = 0
    # prepare search_name:
    if search_name.find(",") > -1:
        snl = search_name.split(",")
        snl.reverse()
        snl = [x.strip() for x in snl]
        search_name = " ".join(snl)
        search_name = search_name.lower()
    params["searchstring"] = search_name
    r = requests.get(getty_url, params=params)
    if r.status_code != 200:
        print(r.status_code, "status")
        return 0
    data = r.json()
    if verbose: 
        print(data)
    query, names, concepts, urls = data
    if not len(names):
        return [[]]
    if len(names) == 1:
        if concepts[0].find(perscon) > -1:
            return [[(names[0], urls[0])]]
    else:
        for idx, concept in enumerate(concepts):
            if concept.find(perscon) > -1:
                uris.append([names[idx], urls[idx]])
    return uris
    

## Function for rendering easydb link javascript

In [129]:
    def mk_link(url, name, linktype="gnd"):
    if not url:
        return url
    url = url.replace("http:", "https:")
    if linktype in ("gnd", "getty)":
        urldict = {"conceptURI": url,
              "conceptName": name}
        return json.dumps(urldict)
    elif linktype == 'wd':
        urldict = {'url': url, 'title': name}
        return json.dumps(urldict)
    return url    

## CSV output

Run for all `Kuenstler` on prod (2023-03-28):

```
not found: 7390
Multiples: 803
Unique: 5567
Too many: 166
found 6536, not found 7390
found 47 % GND entries of 13926 items
CPU times: user 2min 9s, sys: 6.94 s, total: 2min 16s
Wall time: 26min 51s
```

In [ ]:
%%time
#
# 1000 items are processed in 120 secs / 2 mins
#

out = []
total = df.shape[0]
test = total-1
cnt = 0
not_found = []
multiples = []
too_many = []
df.fillna("", inplace=True)
for row in df.itertuples():
    system_object_id = row._4
    ld = row.lebensdaten
    search = row.name
    gnd = query_gnd(search, lebensdaten=ld)
    if not gnd:
        not_found.append((search, system_object_id))
    else:
        if isinstance(gnd, int):
            too_many.append((search, system_object_id, gnd))
            continue
        if len(gnd) > 1:
            multiples.append((search, system_object_id, gnd))
        else:
            out.append((search, system_object_id, gnd))
    cnt += 1
    if cnt > test:
        break
print(f"not found: {len(not_found)}")
# pprint(not_found)
print(f"Multiples: {len(multiples)}")
# pprint(multiples)
print(f"Unique: {len(out)}")
# pprint(out)
print(f"Too many: {len(too_many)}")
print(f"found {len(out)+len(multiples) + len(too_many)}, not found {len(not_found)}")
print("found {} % GND entries of {} items".format(
    round((len(out) + len(multiples) + len(too_many)) * 100 / (total)),
    total))

data = []

# easydb5 fieldnames according to the kuenstler object in the datamodel:
ez5cols = ["_system_object_id", 
           "gnd",
           "gnd_pruefen", 
           "wikidata", 
           "gnd_pruefdaten[].gnd_pruefeintrag"]

for name, sysid, gnd_row in out:
    gnd = gnd_row[0]
    gnd_url = gnd[2]
    gnd_link = mk_link(gnd_url, gnd[0])
    wiki_url = gnd[3]
    wiki_link = ""
    if wiki_url:
        wiki_link = mk_link(wiki_url, gnd[0], linktype="wd")
    tmp = [sysid, gnd_link, 0, wiki_link, ""]
    data.append(tmp)
for name, sysid, gnd_many in too_many:
    data.append([sysid, "", 1, "", ""])
    
for name, sysid, gnd_rows in multiples:
    pruef = []
    for gr in gnd_rows:
        name, birth_year, gnd_url, wd = gr
        pruef.append(mk_link(gnd_url, name))
    tmp = [
        sysid,
        "", 1, "", "\n".join(pruef)
        
    ]
    data.append(tmp)
curt = curtime()
df_gnd = pd.DataFrame(data)
to_csv(df_gnd, f"gnd_kuenstler_{total}_{curt}", headers=ez5cols)
print("use gnd_kuenstler_* csv for importing into csv importer")

## Try out `duckdb` for SQL speed

([duckdb](https://duckdb.org/) offers an easy and fast way to query datasets by SQL)

And look up Getty information...

```
Took 5h 11min on 2023-03-29
6868 unique entries found 
1675 multiple entries
of 13926 objects
````

61% of all entries found.



In [144]:
%%time
from tqdm import notebook, tnrange

first = True #. only show first row
multiples = []
not_found = []
singles = []
datafile = "unidam_kuenstler.zip"
data_csv = fn =  "12476-619740@0678eb41-c907-4f71-a4a1-b9ec119d9652.csv"
if first:
    with ZipFile(str(datadir.joinpath(datafile) )) as zf:
        for fn in zf.namelist():
            if fn.endswith(".csv"):
                print(fn)
                zf.extract(fn)
                break
#lookup = duckdb.sql(f"SELECT _system_object_id, name, " 
#                    f"lebensdaten FROM '{fn}' where lebensdaten <> '' ").fetchall()
lookup = duckdb.sql(f"SELECT _system_object_id, name, " 
                    f"lebensdaten FROM '{fn}'").fetchall()

print(len(lookup), "rows")
total = len(lookup)
pbar = notebook.tqdm(total=total)
for row in lookup:
    if first: 
        print(row)
        break
    sysid, name, bio = row
    if name.find(",") == -1:
        nl = name.rsplit(" ", maxsplit=1)
        nl.reverse()
        name = ", ".join(nl)
    res = query_getty(name, bio, verbose=False)
    pbar.update()
    if res in ([], [[]]):
        not_found.append((sysid, name))
        continue
    if len(res) == 1:
        singles.append((sysid, mk_link(res[0][1], res[0][0])))
    else:
        tmp = []
        for resrow in res:
            tmp.append(mk_link(resrow[1], resrow[0]))
        multiples.append((sysid, "\n".join(tmp)))
if not first:
    cur = curtime()
    ez5cols = ["_system_object_id", "getty"]
    dfgt = pd.DataFrame(singles)
    to_csv(dfgt, f"getty_kuenstler_{len(singles)}_{curt}", headers=ez5cols)
    print(f"found {len(singles)} single items")
    print(f"found {len(multiples)} multiple_items")
    print(f"not found {len(not_found)} items of {len(lookup)}")
    dfgtmu = pd.DataFrame(multiples)
    to_csv(dfgtmu, f"getty_multi_kue_{len(multiples)}_{cur}", headers=["_system_object_id", "getty_multiples"])
    percentage = (len(singles) + len(multiples)) * 100 / total
    print("found {} % Getty entries of {} items".format(
        round(percentage),
        total))


12476-619740@0678eb41-c907-4f71-a4a1-b9ec119d9652.csv
13926 rows


  0%|          | 0/13926 [00:00<?, ?it/s]

(12476, 'Exekias', '')
CPU times: user 400 ms, sys: 102 ms, total: 502 ms
Wall time: 2.9 s


## Toggle `automated` switch

This was forgotten further above, so the field `gnd_getty_auto` is switched to ON

In [167]:
cur = curtime()
headers = ["_system_object_id", "gnd_getty_auto"]
for g in ("gnd", "getty"):
    # select is strange, because columns start with 0,1 as headers in prev. output
    fn = datadir.joinpath("gnd_kuenstler_15000_20230328_0938.csv")
    lookup = duckdb.sql(f"SELECT COLUMNS('0') as '_system_object_id', 1 as 'gnd_getty_auto' FROM '{fn}' OFFSET 1")# .fetchall()
    lookup.write_csv(str(datadir.joinpath(f"autoswitch_upd_{g}_{cur}.csv")), sep=";", header=True)
print("done, use autoswitch_upd_* csv files to update easydb")